### II/ Tìm nghiệm bài toán bằng `TensorFlow`

##### 1) (Full) batch gradient descent: đưa toàn bộ X và Y vào để train:

Với cách 1, do đưa toàn bộ batch vào nên gradient ở mỗi vòng lặp ổn định. Cách này được khuyến khích sử dụng khi hàm cost của mình biết rõ là convex (không có nhiều hơn 1 điểm tối ưu cục bộ). Tuy nhiên, đối với những hàm phức tạp, thì cách 1 có thể ko bao giờ đạt tối ưu toàn cục được.

##### 2) Stochastic gradient descent: đưa từng cặp (x, y) trong data X, Y vào để train :

Đối với cách 2, do mình đưa vào từng cặp nên gradient ở mỗi vòng lặp sẽ rất nhiễu (noisy). Chính vì sự nhiễu này mà có trong qúa trình học, nó có thể giúp mô hình vượt qua được các điểm tối ưu cục bộ. Stochastic = random, thể hiện cho sự nhiễu.
##### 3) Mini-batch gradient descent: bốc 1 lượng nhiều hơn 1 mẫu từ X, Y để train.

Cách 3 là sự kết hợp giữa 1 và 2, cũng là cách dùng nhiều nhất trong deep learning. Trong các bài tới sẽ đề cập sau.

Còn về bài tập thì thực ra hàm error của mình hoàn toàn convex nên dùng cách 1 hay 2 đều được. Nhưng cách 2 sẽ lâu hơn. Bạn có thể sửa code lại để kiểm tra thử.

Đưa dữ liệu vào

In [34]:
from utils_function import load_Boston_housing_data
import numpy as np
raw_train_X, test_X, train_Y, test_Y = load_Boston_housing_data(feature_ind = [4,5])

Selected original features are ['NOX' 'RM']


#### Nhập thư viện

In [35]:
# IMPORT
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import tensorflow as tf
tf.__version__ # '2.x'

'2.3.0'

#### Khai báo biến

In [36]:
learning_rate = 0.005
training_epochs = 10000
display_step = 1000
n_samples, dimension = raw_train_X.shape
batch_size = n_samples # Full Batch Gradient Descent

#### Bài 6. Khai báo

In [37]:
# Implement input and parameter for tensorflow.
train_X = tf.constant(raw_train_X, dtype=tf.float64)

train_Y = tf.reshape(tensor=train_Y, shape=(-1, 1))
train_Y = tf.constant(train_Y, dtype=tf.float64) # convert train_Y to tensor tf

# Set model weights
W = tf.Variable(np.random.normal(size=(dimension, 1)), trainable=True) # create weights variable to train
b = tf.Variable(np.random.normal(size=(1, 1)), trainable=True)
print(W)
print(b)

<tf.Variable 'Variable:0' shape=(2, 1) dtype=float64, numpy=
array([[0.51901381],
       [2.18245458]])>
<tf.Variable 'Variable:0' shape=(1, 1) dtype=float64, numpy=array([[-1.1895192]])>


#### Bài 7. Xây dựng mô hình hồi quy tuyến tính

In [38]:
# GRADED FUNCTION
# TO_DO_6: implement a linear regression function
def tf_lr_hypothesis(X, W, b):
    return tf.add(tf.matmul(X,W), b)

#### Bài 8. Viết hàm cost

In [39]:
# GRADED FUNCTION
# TO_DO_7: implement a cost function
def tf_mse_cost(Y_hat, Y):
    N = len(Y)
    return tf.reduce_sum(tf.pow(Y_hat - Y, 2)) / (2 * N)

#### Bài 9. Viết hàm train

In [40]:
# Create optimizer
optimizer = tf.optimizers.SGD(learning_rate=learning_rate)

#### Bài 10. Chạy chương trình

In [41]:
for epoch in range(training_epochs):
    with tf.GradientTape() as tape:
        Y_hat = tf_lr_hypothesis(train_X, W, b) # apply linear regression function here
        mse_cost = tf_mse_cost(Y_hat, train_Y) # apply mse cost here.
    grads = tape.gradient(mse_cost, [W, b])
    optimizer.apply_gradients(zip(grads, [W, b]))
    if (epoch + 1) % display_step == 0:
        print("Epoch:", epoch + 1, "| Cost:", mse_cost.numpy())

Epoch: 1000 | Cost: 26.81266387034538
Epoch: 2000 | Cost: 24.849733787425997
Epoch: 3000 | Cost: 23.35593886998005
Epoch: 4000 | Cost: 22.219136962141857
Epoch: 5000 | Cost: 21.35399338859174
Epoch: 6000 | Cost: 20.69557260096067
Epoch: 7000 | Cost: 20.194461749699986
Epoch: 8000 | Cost: 19.813059972469727
Epoch: 9000 | Cost: 19.52275473287108
Epoch: 10000 | Cost: 19.301773158895656


####  Bài 11. Tạo các đặc tính mới (Feature Engineering)

In [42]:
X1 = raw_train_X[:,0].reshape((n_samples,1))
X2 = raw_train_X[:,1].reshape((n_samples,1))
X1_sqr = (X1**2).reshape((n_samples,1))
sin_X2 = (np.sin(X2)).reshape((n_samples,1))
X1X2 = (X1*X2).reshape((n_samples,1))
# Create new input from new features
new_train_X = np.concatenate((X1,X2,X1_sqr,sin_X2,X1X2),axis=1) # concatenate new features here

####  Bài 12. Khai báo

In [44]:
# Implement input and parameter for tensorflow.
new_train_X = tf.constant(new_train_X, dtype=tf.float64)
_,new_dimension = new_train_X.shape
optimizer = tf.optimizers.SGD(learning_rate=learning_rate)
train_Y = tf.reshape(tensor=train_Y, shape=(-1, 1))
train_Y = tf.constant(train_Y, dtype=tf.float64) # convert train_Y to tensor tf
training_epochs = 10000
display_step =1000
# set model weight
W = tf.Variable(np.random.normal(size=(new_dimension, 1)), trainable=True) # create weights variable to train
b = tf.Variable(np.random.normal(size=(1, 1)), trainable=True)

####  Bài 13. Chạy chương trình

In [45]:
for epoch in range(training_epochs):
    with tf.GradientTape() as tape:
        Y_hat = tf_lr_hypothesis(new_train_X, W, b) # apply linear regression function here
        mse_cost = tf_mse_cost(Y_hat, train_Y) # apply mse cost here
    grads = tape.gradient(mse_cost, [W, b])
    optimizer.apply_gradients(zip(grads, [W, b]))
    if (epoch + 1) % display_step == 0:
        print("Epoch:", epoch + 1, "| Cost:", mse_cost.numpy())


Epoch: 1000 | Cost: 18.39951659496526
Epoch: 2000 | Cost: 18.248048749352442
Epoch: 3000 | Cost: 18.2362454396768
Epoch: 4000 | Cost: 18.229800229251122
Epoch: 5000 | Cost: 18.223911257440697
Epoch: 6000 | Cost: 18.21810898150335
Epoch: 7000 | Cost: 18.21233940723464
Epoch: 8000 | Cost: 18.20659613368258
Epoch: 9000 | Cost: 18.200878288968998
Epoch: 10000 | Cost: 18.195185645256906
